In [13]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [14]:
x_range = (0.0, 10)  # Domain for the ODE
initial_conditions = [
    {'type': 'y', 'value': 1.0, 'x': 0.0},    # y(0) = 1
    {'type': 'y\'', 'value': 0.0, 'x': 0.0},  # y'(0) = 0
]

learning_rate = 0.0001
num_epochs = 10000

In [15]:
class Network(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Network, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, H)
        self.linear3 = torch.nn.Linear(H, H)
        self.linear4 = torch.nn.Linear(H, D_out)
        
        # Initialize biases and weights
        torch.nn.init.constant_(self.linear1.bias, 0.)
        torch.nn.init.constant_(self.linear2.bias, 0.)
        torch.nn.init.constant_(self.linear3.bias, 0.)
        torch.nn.init.constant_(self.linear4.bias, 0.)
        
        torch.nn.init.normal_(self.linear1.weight, mean=0, std=0.1)
        torch.nn.init.normal_(self.linear2.weight, mean=0, std=0.1)
        torch.nn.init.normal_(self.linear3.weight, mean=0, std=0.1)
        torch.nn.init.normal_(self.linear4.weight, mean=0, std=0.1)

    def forward(self, x):
        y1 = torch.tanh(self.linear1(x))
        y2 = torch.tanh(self.linear2(y1))
        y3 = torch.tanh(self.linear3(y2))
        y = self.linear4(y3)
        return y

In [16]:
D_in, H, D_out = 1, 1000, 1  # Example dimensions, can be modified based on ODE requirements
model = Network(D_in, H, D_out)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [17]:
def energy_functional(x, y, dy_dx):
    # Defines the energy functionalor aka the ODE/PDE based on the problem requirements. 
    # Here you can modify the functino yourself to match the energy function of your specific problem.
    # Like as an examle, if it's elasticity, you can use function of strain energy
    # Here, I defined a dummy function for simple demonstration.
    return 0.5 * (dy_dx ** 2) - 0.5 *  (y ** 2)  # Example: strain energy with linear potential that I found online.

In [18]:
# x_range = (0.0, 10)  # Domain for the ODE
# initial_conditions = [
#     {'type': 'y', 'value': 1.0, 'x': 0.0},    # y(0) = 1
#     {'type': 'y\'', 'value': 0.0, 'x': 0.0},  # y'(0) = 0
# ]

# learning_rate = 0.001
# num_epochs = 1000

In [19]:
def compute_energy_integral(model, x_range, num_points=1000):
    """Compute the integral of the energy functional over the specified x range."""
    x_min, x_max = x_range
    x = torch.linspace(x_min, x_max, num_points).view(-1, 1)
    x.requires_grad = True  
    
    y_pred = model(x)
    dy_dx_pred = torch.autograd.grad(y_pred, x, torch.ones_like(y_pred), create_graph=True)[0]
    
    energy_vals = energy_functional(x, y_pred, dy_dx_pred)
    
    integral = torch.mean(energy_vals) * (x_max - x_min)
    return integral ** 2

In [20]:
def apply_initial_conditions(model, initial_conditions):
    ivp_residuals = []
    for ivp in initial_conditions:
        x_ivp = torch.tensor([[ivp['x']]], requires_grad=True)
        y_ivp = model(x_ivp)
        
        if ivp['type'] == 'y':
            ivp_residual = y_ivp - ivp['value']
        elif ivp['type'] == 'y\'':
            dy_dx_ivp = torch.autograd.grad(y_ivp, x_ivp, torch.ones_like(x_ivp), create_graph=True)[0]
            ivp_residual = dy_dx_ivp - ivp['value']
        
        ivp_residuals.append(ivp_residual ** 2)
    
    return torch.sum(torch.stack([ivp**2 for ivp in ivp_residuals]))

In [21]:
def train(model, optimizer, x_range, initial_conditions, num_epochs):
    for epoch in range(num_epochs):
        optimizer.zero_grad()

        # Compute the energy integral
        energy_integral = compute_energy_integral(model, x_range)

        # Apply initial condition residuals as constraints
        ivp_residual = apply_initial_conditions(model, initial_conditions)

        # Total loss: Energy integral + initial condition penalty
        loss = energy_integral + 500 * ivp_residual # ** 2 
        loss.backward()
        optimizer.step()
        # scheduler.step()
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
train(model, optimizer, x_range, initial_conditions, num_epochs)

Epoch 0, Loss: 35825.34375
Epoch 100, Loss: 7.348491191864014
Epoch 200, Loss: 3.1059913635253906
Epoch 300, Loss: 1.463247299194336
Epoch 400, Loss: 0.7590457797050476
Epoch 500, Loss: 0.42577511072158813
Epoch 600, Loss: 0.25313085317611694
Epoch 700, Loss: 0.15703243017196655
Epoch 800, Loss: 0.10054323077201843
Epoch 900, Loss: 0.06594889611005783
Epoch 1000, Loss: 0.044068269431591034
Epoch 1100, Loss: 0.0298821609467268
Epoch 1200, Loss: 0.020488539710640907
Epoch 1300, Loss: 0.01416288036853075
Epoch 1400, Loss: 0.009852958843111992
Epoch 1500, Loss: 0.006889820098876953
Epoch 1600, Loss: 0.00484743295237422
Epoch 1700, Loss: 0.003431899705901742
Epoch 1800, Loss: 0.0024412234779447317
Epoch 1900, Loss: 0.001745328539982438
Epoch 2000, Loss: 0.0012551441323012114
Epoch 2100, Loss: 0.0009091426618397236
Epoch 2200, Loss: 0.000663889164570719
Epoch 2300, Loss: 0.0004914018791168928
Epoch 2400, Loss: 0.00036884163273498416
Epoch 2500, Loss: 0.00028070586267858744
Epoch 2600, Loss: 

In [ ]:
def visualize_solution(model, x_range):
    x_min, x_max = x_range
    x_values = np.linspace(x_min, x_max, 100)
    x_tensor = torch.tensor(x_values, dtype=torch.float32).view(-1, 1)
    y_pred = model(x_tensor).detach().numpy()

    plt.plot(x_values, y_pred, label="DEM Solution")
    plt.xlabel("x")
    plt.ylabel("y(x)")
    plt.title("Solution of the Harmonic Oscillator via DEM")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
visualize_solution(model, x_range)